In [1]:
import pandas as pd
import warnings, requests
from pathlib import Path
import fuzzymatcher

In [3]:
def ag_get_data(query):
    warnings.simplefilter('ignore', requests.packages.urllib3.exceptions.InsecureRequestWarning)
    params = {"Database": "OIL", "Query": query}
    resp = requests.post('https://TST-QDEV-AP1.petroineos.local:5001/genericdata/Fetch/', json=params, verify=False)
    json_result = resp.json()
    df = pd.DataFrame(json_result)
    return df

In [ ]:
# genscape = ag_get_data('select * from OIL.Refinery.RefineryUnitDailyStatus')

In [ ]:
# mapping = ag_get_data('select * from OIL.Refinery.RefineryUnitMapping')

In [ ]:
# iir = ag_get_data('select * from OIL.Refinery.MaintenanceDaily')

In [ ]:
# iir_mapping = ag_get_data('select * fromOIL.Refinery.RefineryUnit')

In [16]:
gen_units = ag_get_data(
    '''
  Select FacilityId, UnitId, FacilityName, UnitName, UnitCapacity, Region, ProcessingClass, UnitCategory from oil.Refinery.RefineryUnitDailyStatus
  group by FacilityId, UnitId, FacilityName, UnitName, UnitCapacity, Region, ProcessingClass, UnitCategory
    '''
)

In [17]:
gen_units.head()

FacilityId                         FacilityName     ProcessingClass  \
0         AA                 Valero - Port Arthur  Primary Processing   
1         AA                 Valero - Port Arthur  Primary Processing   
2         AA                 Valero - Port Arthur      Light Products   
3         AB                   Lyondell - Houston  Primary Processing   
4         AC  Flint Hills - Corpus Christi (West)  Primary Processing   

       Region  UnitCapacity                    UnitCategory UnitId UnitName  
0  Gulf Coast      150000.0  Atmospheric Crude Distillation    AA1    CDU 1  
1  Gulf Coast       48000.0       Vacuum Crude Distillation    AA3    VDU 1  
2  Gulf Coast       80000.0         Fluid Catalytic Cracker    AA5    FCC 1  
3  Gulf Coast      138000.0  Atmospheric Crude Distillation    AB2    CDU 2  
4  Gulf Coast      180000.0  Atmospheric Crude Distillation    AC1    CDU 1

In [20]:
gen_duplicates = gen_units[gen_units.duplicated(subset='UnitId', keep=False)].copy()
gen_duplicates.sort_values(by='UnitId', inplace=True)

In [22]:
gen_duplicates.head()

FacilityId          FacilityName     ProcessingClass      Region  \
0           AA  Valero - Port Arthur  Primary Processing  Gulf Coast   
28          AA  Valero - Port Arthur  Primary Processing  Gulf Coast   
234         AA  Valero - Port Arthur  Primary Processing  Gulf Coast   
235         AA  Valero - Port Arthur  Primary Processing  Gulf Coast   
205         AB    Lyondell - Houston  Primary Processing  Gulf Coast   

     UnitCapacity                    UnitCategory UnitId UnitName  
0        150000.0  Atmospheric Crude Distillation    AA1    CDU 1  
28        75000.0  Atmospheric Crude Distillation    AA1    CDU 1  
234      265000.0  Atmospheric Crude Distillation    AA2    CDU 2  
235      268000.0  Atmospheric Crude Distillation    AA2    CDU 2  
205      140000.0  Atmospheric Crude Distillation    AB1    CDU 1

In [67]:
gen_id = ag_get_data('Select FacilityId, UnitId from oil.Refinery.RefineryUnitDailyStatus group by FacilityId, UnitId')
iir_id = ag_get_data('select UNIT_ID, PLANT_ID from oil.Refinery.RefineryUnit iir')
mapping = ag_get_data('select * from [OIL].[Refinery].[RefineryUnitMapping]')
iir_names = ag_get_data('select UNIT_ID, PLANT_ID, UNIT_NAME, PLANT_NAME from oil.Refinery.RefineryUnit iir')

In [43]:
len(mapping)

268

In [46]:
iir_mapped.head()

PLANT_ID  UNIT_ID    Vendor VendorValue
0  1021435  1011144  Genscape        AT12
1  1021435  1011146  Genscape         AT3
2  1021435  1011147  Genscape         AT5
3  1024611  1019904  Genscape         DY1
4  1024611  1019905  Genscape         DY2

In [48]:
iir_mapped = iir_id.merge(mapping, how='inner', on='UNIT_ID')
iir_gen = iir_mapped.merge(gen_id, how='inner', left_on='VendorValue', right_on='UnitId')

In [51]:
len(iir_gen)

267

In [62]:
ref_mapping = iir_gen[['PLANT_ID', 'Vendor', 'FacilityId']].drop_duplicates()
len(ref_mapping)

49

In [65]:
# ref_mapping.to_excel('iir_genscape_refinery_mapping.xlsx')

In [69]:
iir_refs = iir_names[['PLANT_ID', 'PLANT_NAME']].drop_duplicates()

In [71]:
len(iir_refs)

731

In [73]:
ref_mapping_name = ref_mapping.merge(iir_refs, how='left', on='PLANT_ID')

In [75]:
# ref_mapping_name

In [31]:
# gen_iir = ag_get_data(
#     '''
#     with gen(facility_id, unit_id) as
#     (Select FacilityId, UnitId from oil.Refinery.RefineryUnitDailyStatus
#     group by FacilityId, UnitId)

#     select * from oil.Refinery.RefineryUnit iir
#     join [OIL].[Refinery].[RefineryUnitMapping] map
#     on map.UNIT_ID=iir.UNIT_ID
#     join gen on gen.unit_id=map.VendorValue
#     '''
# )

In [36]:
# iir_map = ag_get_data(
# '''
# select * from oil.Refinery.RefineryUnit iir
# join [OIL].[Refinery].[RefineryUnitMapping] map
# on map.UNIT_ID=iir.UNIT_ID
# join gen on gen.unit_id=map.VendorValue
# '''
# )

In [76]:
iir_refs = ag_get_data(
'''
select PLANT_ID, PLANT_NAME, OWNER_NAME, COUNTRY, sum(U_CAPACITY) CDU from oil.Refinery.RefineryUnit iir
where UTYPE_DESC='Atmospheric Distillation'
group by PLANT_ID, PLANT_NAME, OWNER_NAME, COUNTRY
'''
)

In [81]:
iir_country_mapping = pd.read_excel('//petroineos.local/DFS/Profiles/syedahmad/Desktop/mapping_ext.xlsx', sheet_name='IIR country mapping')
country_mapping = pd.read_excel('//petroineos.local/DFS/Profiles/syedahmad/Desktop/mapping_ext.xlsx', sheet_name='country mapping')

In [83]:
country_mapping_short = country_mapping[['alpha-2', 'region', 'sub-region']].copy()

In [86]:
iir_country_mapping.head()

country_iir country_mapped
0          Finland             FI
1       Martinique             MQ
2          Bahrain             BH
3  Myanmar (Burma)             MM
4        Macedonia             MK

In [95]:
iir_with_code = iir_refs.merge(iir_country_mapping, how='left', left_on='COUNTRY', right_on='country_iir')
iir_region = iir_with_code.merge(country_mapping_short, how='left', left_on='country_mapped', right_on='alpha-2')
iir_region.drop(columns=['COUNTRY', 'alpha-2'], inplace=True)

In [97]:
iir_region.columns

Index(['CDU', 'OWNER_NAME', 'PLANT_ID', 'PLANT_NAME', 'country_iir',
       'country_mapped', 'region', 'sub-region'],
      dtype='object')

In [99]:
iir_region[['region', 'sub-region','country_iir','country_mapped','OWNER_NAME', 'PLANT_ID', 'PLANT_NAME','CDU']].to_excel('iir_refs.xlsx')

### ------------------------ IIR and woodmac fuzzy matching --------------------

In [39]:
iir = pd.read_excel('iir_refs.xlsx', sheet_name='Sheet1')
woodmac_units = pd.read_excel('RefineryProfile_2021-09-22_16-29-02.844.xlsx', sheet_name='Refinery Capacities (kbsd)', header=9)
manual_map = pd.read_excel('woodmac_iir_mapping_manual.xlsx', sheet_name='Refinery Capacities (kbsd)', header=9)

In [5]:
woodmac_capacity = woodmac_units[['Refinery', 'CDU']].copy()

In [6]:
woodmac_owners = pd.read_excel('RefineryProfile_2021-09-22_16-29-02.844.xlsx', sheet_name='Refinery Ownership', header=9)
woodmac_owners.sort_values(by=['Refinery', 'Interest (%)'], ascending=False,inplace=True)
woodmac_owners = woodmac_owners.drop_duplicates(subset='Refinery',keep='first')
woodmac_owners = woodmac_owners[['Refinery', 'Owner']].copy()

In [26]:
woodmac = woodmac_owners.merge(woodmac_capacity, how='outer', on='Refinery')

In [31]:
iir['PLANT_NAME'] = iir['PLANT_NAME'].str.replace('Refinery', '')
iir['PLANT_NAME'] = iir['PLANT_NAME'].str.strip()

In [34]:
left = ['Refinery']
right = ['PLANT_NAME']
matched = fuzzymatcher.fuzzy_left_join(woodmac,iir,left_on=left, right_on=right, right_id_col='PLANT_ID')
mapping = matched.sort_values(by='best_match_score', ascending=False)

In [35]:
left = ['Refinery', 'Owner']
right = ['PLANT_NAME', 'OWNER_NAME']
matched_owner = fuzzymatcher.fuzzy_left_join(woodmac,iir,left_on=left, right_on=right, right_id_col='PLANT_ID')
owner_mapping = matched_owner.sort_values(by='best_match_score', ascending=False)

In [36]:
left = ['Refinery', 'Owner', 'CDU']
right = ['PLANT_NAME', 'OWNER_NAME', 'cdu']
matched_cdu = fuzzymatcher.fuzzy_left_join(woodmac,iir,left_on=left, right_on=right, right_id_col='PLANT_ID')
cdu_mapping = matched_cdu.sort_values(by='best_match_score', ascending=False)

In [37]:
mapping.to_excel('mapping.xlsx')
owner_mapping.to_excel('mapping_owner_name.xlsx')
cdu_mapping.to_excel('mapping_owner_name_cdu.xlsx')

In [40]:
manual_map.head()

Region  iir_plant_id  Country        Refinery Refinery Status  \
0  Africa     3308936.0  Algeria           Adrar     Operational   
1  Africa     3028983.0  Algeria         Algiers     Operational   
2  Africa     1060435.0  Algeria           Arzew     Operational   
3  Africa     3028990.0  Algeria  Hassi Messaoud     Operational   
4  Africa     1087049.0  Algeria          Skikda     Operational   

  Capacity Date       CDU     VDU  CCU/FCC     RCC  ...  CRF  CTL - Indirect  \
0    2021-01-01   13000.0     NaN      NaN  5500.0  ...  NaN             NaN   
1    2021-01-01   74000.0     NaN      NaN     NaN  ...  NaN             NaN   
2    2021-01-01   75000.0  5100.0      NaN     NaN  ...  NaN             NaN   
3    2021-01-01   26600.0     NaN      NaN     NaN  ...  NaN             NaN   
4    2021-01-01  330000.0  5100.0      NaN     NaN  ...  NaN             NaN   

   H2 (Mscf/d)  IGCC (MW)     LUB  Naphtha Splitter  RESID GASIF  \
0          NaN        NaN     NaN               NaN          NaN   
1          NaN        NaN     NaN               NaN          NaN   
2          NaN        NaN  3200.0               NaN          NaN   
3          NaN        NaN     NaN               NaN          NaN   
4          NaN        NaN     NaN               NaN          NaN   

   Sulphur Recovery  THERM  VSB  
0               NaN    NaN  NaN  
1               NaN    NaN  NaN  
2               NaN    NaN  NaN  
3               NaN    NaN  NaN  
4               NaN    NaN  NaN  

[5 rows x 48 columns]

In [45]:
mapping_merged_owner = owner_mapping.merge(manual_map[['Refinery', 'iir_plant_id']], how='left', on='Refinery')
mapping_merged = mapping.merge(manual_map[['Refinery', 'iir_plant_id']], how='left', on='Refinery')

In [46]:
mapping_merged.to_excel('merged_mapping.xlsx')
mapping_merged_owner.to_excel('merged_mapping_owner.xlsx')

In [ ]:
#filter IIR table for only emea data